In [18]:
import numpy as np
import json
import os, sys, pathlib
import matplotlib
import matplotlib.pyplot as plt

In [19]:
bench_path = pathlib.Path("./output/benchmark/")
bench_path

PosixPath('output/benchmark')

In [20]:
experiments = list(bench_path.glob("*.json"))

In [27]:
c10_exp = []
for exp in experiments:
#     print(exp.name)
    if '_nPE_cifar100_' in exp.name and not 'sMlp' in exp.name:
        c10_exp.append(exp)
c10_exp

[PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch2_l4_att_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch8_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch4_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch2_l4_sAtt_mlp_s147_rand_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch2_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch4_l8_att_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch1_l4_sAtt_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch8_l4_att_mlp_s147_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch4_l4_sAtt_mlp_s147_rand_data.json'),
 PosixPath('output/benchmark/01.3_ViT_nPE_cifar100_patch4_l4_att_mlp_s147_data.json')]

In [28]:
import pandas as pd

In [29]:
exp_dict = {'ps':[], 'la':[], 'sAtt':[], 'acc':[], 'mem':[], 'time':[], 'params':[]}
for file in c10_exp:
    ## filter out name from filename
    fn = file.name
    splits = fn.split('_')
    ps = int(splits[4][5:])
    la = int(splits[5][1:])
    sAtt = (splits[6] == 'sAtt')
    
    #############################
    ## Remove this to ignore random tokens experiments and use normal experiment
    if sAtt:
        if not (fn.endswith('_rand_data.json')):
            continue
    #############################
    
    with open(file, 'r') as f:
        data = json.load(f)
        
    best_acc = np.array(data['test_stat'])[:, 2].max()
    params = data['params']
    
#     print(ps, la, sAtt,  best_acc)
    exp_dict['ps'].append(ps)
    exp_dict['la'].append(la)    
    exp_dict['sAtt'].append(sAtt)
    exp_dict['acc'].append(best_acc)
    exp_dict['params'].append(params)
    
    ### search for benchmark memory and time in a table
    with open("./output/benchmark_memory_data.json", 'r') as f:
        data2 = json.load(f)
        
    mem, time = -1, -1
    try:
        data2 = data2[fn[:-10]]
        mem = data2['memory']
        time = data2['time_train']*1000
    except:
        pass
    exp_dict['mem'].append(mem)
    exp_dict['time'].append(time)
    
exp_df = pd.DataFrame(exp_dict)
exp_df.sort_values('ps', axis=0)

,ps,la,sAtt,acc,mem,time,params
0,2,4,False,49.64,976,12.911382,1773220
1,2,4,True,48.30,-1,-1.000000,1773220
2,4,8,False,29.18,-1,-1.000000,1885412
4,4,4,True,53.63,-1,-1.000000,1355492
5,4,4,False,53.20,478,10.480094,1355492
3,8,4,False,51.21,-1,-1.000000,2567524


In [12]:
# with open(filename, 'r') as f:
#     file_data = json.load(f)

NameError: name 'filename' is not defined